# Case study: Reed breast T cell data

This notebook generates results for a case study that uses a [Human Breast Cell Atlas dataset](https://cellxgene.cziscience.com/collections/48259aa8-f168-4bf5-b797-af8e88da6637) (T cell subset).

This is an example where we use the Extended Neighbourhood-Proportion-Error (xNPE) to compare embedding distortion of different cell populations, as well as neighbourhood composition plots to reveal sources of distortion.

<hr>

## **0.** Load required modules

In addition to `ViVAE`, `ViScore` and their dependenceis, you will need to have `umap` installed in your conda environment.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import copy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import ViVAE
import ViScore
from sklearn.manifold import TSNE
from umap import UMAP

## **1.** Load input data

See the ViScore [benchmarking](https://github.com/saeyslab/ViScore/tree/main/benchmarking) page for instructions on how to obtain and pre-process the Reed data.

In [ ]:
## Load input data

dataset = 'Reed'

pc    = np.load(os.path.join('..', 'data', f'{dataset}_input.npy'),          allow_pickle=True)
pc_d  = np.load(os.path.join('..', 'data', f'{dataset}_input_denoised.npy'), allow_pickle=True)
labs  = np.load(os.path.join('..', 'data', f'{dataset}_labels.npy'),         allow_pickle=True)
unass = np.load(os.path.join('..', 'data', f'{dataset}_unassigned.npy'),     allow_pickle=True).item()

We will also generate (or load) a *k*-nearest-neighbour graph for 1000 neighbours, to use for the xNPE computation.

In [ ]:
knn1000 = ViScore.make_knn(pc, fname=os.path.join('..', 'data', f'{dataset}_knn1000.npy'), k=1000, random_state=42)

## **2.** Generating embeddings

We use *t*-SNE, UMAP and ViVAE to generate embeddings of this dataset.
To prevent any non-coverged results, we set the iteration/epoch count for each method higher than default here...

In [ ]:
model_tsne = TSNE(n_components=2, n_iter=2000, random_state=42)
emb_tsne = model_tsne.fit_transform(pc)

model_umap = UMAP(n_components=2, n_epochs=1000, random_state=42)
emb_umap = model_umap.fit_transform(pc)

model_vivae = ViVAE.ViVAE(input_dim=pc.shape[1], latent_dim=2, random_state=42)
model_vivae.fit(pc_d, batch_size=1024, n_epochs=100)
emb_vivae = model_vivae.transform(pc_d)

## **3.** Compute population-level xNPE scores

In [ ]:
xnpe_tsne = ViScore.xnpe(hd=pc, ld=emb_tsne, annot=labs, knn=knn1000, random_state=42)
xnpe_umap = ViScore.xnpe(hd=pc, ld=emb_umap, annot=labs, knn=knn1000, random_state=42)
xnpe_vivae = ViScore.xnpe(hd=pc, ld=emb_vivae, annot=labs, knn=knn1000, random_state=42)

In [ ]:
palette = ['#726ca6',
 '#8ff56b',
 '#79d0f9',
 '#fba56a',
 '#eefc85',
 '#aeaead',
 '#6e85ff',
 '#b97671',
 '#dbbafd',
 '#6bb277',
 '#b7fbce',
 '#6af1b0',
 '#b26ae7',
 '#fb6c98',
 '#fdc4b8',
 '#c1c86c',
 '#699dc0',
 '#d889c1',
 '#a89ef4',
 '#95d598',
 '#757469',
 '#78fefe',
 '#f1f7c3',
 '#b2ddfb',
 '#cad9a3',
 '#9b9f69',
 '#aa7caa',
 '#74c7c0',
 '#face7e',
 '#fe9cdb',
 '#ce9f81',
 '#bafc85',
 '#fdd5f1',
 '#e97f6c',
 '#8d89d7',
 '#839095',
 '#d68ef9',
 '#a5d1ca',
 '#d1fcfe',
 '#6eaaee',
 '#f799a1',
 '#d7b2c8',
 '#70d87a',
 '#99faa9',
 '#6b70d4',
 '#dbd8d2',
 '#fb77c9',
 '#88f4d2',
 '#d17a98',
 '#90b2d3',
 '#aafef7',
 '#debc9d',
 '#d2e96a',
 '#96c96a',
 '#8c6ff5',
 '#927286',
 '#7cff8e',
 '#80b19e',
 '#adbcfa',
 '#d86fdd',
 '#aee276',
 '#eee1a5',
 '#feb6fe',
 '#996dc9',
 '#b699cc',
 '#ad908d',
 '#76946b',
 '#d2fea8',
 '#a7b883',
 '#b881fe',
 '#69e7da',
 '#92e8f5',
 '#b5b6d4',
 '#dadcfa',
 '#bf6cbe',
 '#9199b6',
 '#70d79f',
 '#6afd6e',
 '#dcb26c',
 '#d69fae',
 '#b5eab1',
 '#fce96a',
 '#6987aa',
 '#8dadfc',
 '#938afd',
 '#c7ebe4',
 '#de6a7f',
 '#938669',
 '#c4cce8',
 '#e36daf',
 '#e8f1e8',
 '#86e1b6',
 '#ff6b69',
 '#ed9ffc',
 '#87d7d6',
 '#feb58d',
 '#b96a93',
 '#dcd189',
 '#adc9a7']

In [ ]:
mpl.rcParams['axes.linewidth'] = 0.1
pops = list(xnpe_tsne.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(5, 1.5), dpi=150)
fig.subplots_adjust(wspace=.3)
handles = []
labels = []
embs = [emb_tsne, emb_umap, emb_vivae]
s = .01
for i, method in enumerate(methods):
        idcs0 = np.where(labs==unass)[0]
        idcs1 = np.delete(np.arange(len(labs)), idcs0)
        emb = embs[i]
        ax[i].scatter(emb[idcs0,0], emb[idcs0,1], s=s, c='#bfbfbf', alpha=1., linewidths=0)
        idx_pop = 0
        l = np.delete(labs, idcs0)
        for pop in np.unique(l):
            idcs = np.where(l == pop)[0]
            if i==(len(methods)-1):
                ax[i].scatter(emb[idcs1[idcs],0], emb[idcs1[idcs],1], label=pop, s=s, c=palette[idx_pop], alpha=1., linewidth=.5)
            else:
                ax[i].scatter(emb[idcs1[idcs],0], emb[idcs1[idcs],1], s=s, c=palette[idx_pop], alpha=1., linewidth=.5)
            ax[i].tick_params(axis='both', labelsize=5)
            idx_pop += 1
        ax[i].axis('equal')
        ax[i].set_title(methods[i], size=7, ha='left', x=-.03, y=.98)
        if i==(len(methods)-1):
            l = fig.legend(bbox_to_anchor=(1.35, .92), fontsize=5, markerscale=50.)
fig.savefig('Reed_A_embeddings.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('Reed_A_embeddings.svg', bbox_inches='tight', transparent=True)

## **4.** Create neighbourhood composition plots

Next, we set a population of interest (`poi`: `'mature NK T cell`), highlight it in the embeddings and generate neighbourhood composition plots that show sources of the error for this POI.

In [ ]:
poi = 'mature NK T cell'

In [ ]:
mpl.rcParams['axes.linewidth'] = 0.1
pops = list(xnpe_tsne.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(5, 1.5), dpi=150)
fig.subplots_adjust(wspace=.3)
handles = []
labels = []
embs = [emb_tsne, emb_umap, emb_vivae]
s = .01

idcs_nk = np.where(labs==poi)[0]
for i, method in enumerate(methods):

        emb = embs[i]
        ax[i].scatter(emb[:,0], emb[:,1], s=s, c='grey', alpha=1., linewidth=.5)
        ax[i].scatter(emb[idcs_nk,0], emb[idcs_nk,1], s=3, c=palette[idx_pop], alpha=1., linewidth=.5)

        ax[i].axis('equal')
        ax[i].set_title(methods[i], size=7, ha='left', x=-.03, y=.98)
        ax[i].tick_params(axis='both', labelsize=5)
fig.savefig('reed_B_embeddings_poi.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_B_embeddings_poi.svg', bbox_inches='tight', transparent=True)

In [ ]:
pops = list(xnpe_tsne.keys())
methods = ['t-SNE', 'UMAP', 'ViVAE']
nmet = len(methods)
d = pd.DataFrame(
    data=np.vstack([list(xnpe_tsne.values()), list(xnpe_umap.values()), list(xnpe_vivae.values())]),
    columns=pops
)
d.index = methods
d = d.drop(unass, axis=1)
pops = list(d.keys())
npop = len(pops)
d

In [ ]:
fig, ax = plt.subplots(figsize=(2,4), dpi=150)
mpl.rcParams['axes.linewidth'] = 0.1
pal = ['orange', 'teal', 'darkred']
n = npop*(nmet+1)

for i, method in enumerate(methods):
    for j, pop in enumerate(pops):
        if j==0:
            ax.barh(y=j*(nmet+1)+i, width=d[pop][method], color=pal[i], label=method)
        else:
            ax.barh(y=j*(nmet+1)+i, width=d[pop][method], color=pal[i])
ax.set_xlim((0., 1.2))
ax.set_ylim((-1, n))
pop_labels = copy.deepcopy(pops)
ax.set_yticks(ticks=np.arange(npop)*(nmet+1)+1, labels=pop_labels)
ax.tick_params(axis='both', labelsize=6)
plt.gca().invert_yaxis()

fig.legend(bbox_to_anchor=(.88, .845), fontsize=6)
fig.savefig('reed_C_barplot.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_C_barplot.svg', bbox_inches='tight', transparent=True)

In [ ]:
pop = poi
nc_hd = ViScore.neighbourhood_composition(X=pc, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_tsne = ViScore.neighbourhood_composition(X=emb_tsne, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_umap = ViScore.neighbourhood_composition(X=emb_umap, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)
nc_vivae = ViScore.neighbourhood_composition(X=emb_vivae, pop=pop, annot=labs, k=1000, stepsize=10, random_state=42)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=4, figsize=(3, 5), dpi=150, sharey=True, sharex=True)
mpl.rcParams['axes.linewidth'] = 0.1
fig.subplots_adjust(hspace=.35)

for i, nc in enumerate([nc_hd, nc_tsne, nc_umap, nc_vivae]):
    y = np.vstack(nc[3])
    ax[i].stackplot(nc[4], y.T, labels=nc[1], colors=palette[1:])
    ax[i].set_ylim((0.,1.))
    ax[i].set_xlim((min(nc[4]), max(nc[4])))
    ax[i].set_title(['HD data', 't-SNE', 'UMAP', 'ViVAE'][i], size=7, ha='left', x=.0, y=.96)
    ax[i].tick_params(axis='both', labelsize=5)

ax[3].set_ylabel('Neighbour proportion', loc='bottom', size=7)
ax[3].set_xlabel('Neighbourhood rank', loc='left', size=7)

fig.savefig('reed_D_ncp.png', bbox_inches='tight', dpi=300, transparent=True)
fig.savefig('reed_D_npc.svg', bbox_inches='tight', transparent=True)